#### <b>Tutorial 1

Review of the Script:

- omegaconf is installed by default with hydra. It is only used to provide the type annotation for cfg argument in func.
- @hydra.main(config_path="config", config_name="config") This is the main decorator function that is used when any function requires contents from a configuration file.
- Current working directory is changed. main.py exists in src/main.py but the output shows the current working directory is src/outputs/2021-03-13/16-22-21. This is the most important point when using Hydra. An explanation follows below.

#### How Hydra handles different runs

Whenever a program is executed using python main.py, Hydra will create a new folder in outputs directory with the following naming scheme outputs/YYYY-mm-dd/HH-MM-SS i.e. the date and time at which the file was executed. Think about this for a second. Hydra provides you a way to maintain a log of every run without you having to worry about it.

```
src
├── config
│   └── config.yaml
├── main.py
├── outputs
│   └── 2021-03-13
│       └── 17-14-24
│           ├── .hydra
│           │   ├── config.yaml
│           │   ├── hydra.yaml
│           │   └── overrides.yaml
│           ├── main.log
```

What happens actually? When you run src/main.py, hydra moves this file to src/outputs/2021-03-13/16-22-21/main.py and then runs it. 

- You can verify this by checking the output of os.getcwd() as shown in the above example. 
- This means if your main.py relied on some external file, say test.txt, then you would have to use ../../../test.txt instead, as you are no longer running the program in src directory. 
- This also means that everything you save to disk would be saved relative to src/outputs/2021-03-13/16-22-21/.

Hydra provides two utility functions to handle this situation

- <b>hydra.utils.get_original_cwd()</b>: Get the original current working directory i.e. `src`.

```
orig_cwd = hydra.utils.get_original_cwd()
path = f"{orig_cwd}/test.txt"
# path = src/test.txt
```

- <b>hydra.utils.to_absolute_path(file_name):</b>

```
path = hydra.utils.to_absolute_path('test.txt')

# path = src/test.txt
```

Suppose we want to read src/test.txt and write the output to output.txt. The corresponding function to do this would be as shown below:

```
@hydra.main(config_path="config", config_name="config")
def func(cfg: DictConfig):
    orig_cwd = hydra.utils.get_original_cwd()

    # Read file
    path = f"{orig_cwd}/test.txt"
    with open(path, "r") as f:
        print(f.read())

    # Write file
    path = f"output.txt"
    with open(path, "w") as f:
        f.write("This is a dog")
```

After running the above code, we check the directory structure again:

```
src
├── config
│   └── config.yaml
├── main.py
├── outputs
│   └── 2021-03-13
│       └── 17-14-24
│           ├── .hydra
│           │   ├── config.yaml
│           │   ├── hydra.yaml
│           │   └── overrides.yaml
│           ├── main.log
│           └── output.txt
└── test.txt
```

The file was written to the folder created by hydra. 

- This is a good way to save intermediate results when you are developing something. 
- You can use this feature to save the accuracy results of your model with different hyperparameters. 
- Now you do not have to spend time on manually saving the configuration file or the command line arguments you used to run the script and creating a new folder for each run to store the outputs.

#### Contents of Subfolder in Outputs Folder (Hydra)

Each subfolder have the following substructure:

```
src/outputs/2021-03-13/17-14-24/
├── .hydra
│   ├── config.yaml
│   ├── hydra.yaml
│   └── overrides.yaml
└── main.log
```

- `config.yaml` - Copy of the config file passed to the function (It doesn't matter if you pass foo.yaml, this file would still be named config.yaml)
- `hydra.yaml` - Copy of the hydra config file. We will later see how to change some of the defaults used by hydra. (You can specify the message of python main.py --help here)
- `overrides.yaml` - Copy of any argument that you provide through the command line and which changes one of the default value would be stored here
- `main.log` - Output of the logger would be stored here. (For foo.py this file would be named foo.log)

#### How to use Logging

With Hydra, you can easily use the logging package provided by Python in your code without any setup. The output of the log is stored in main.log. A usage example is shown below:

```
import logging

log = logging.getLogger(__name__)

@hydra.main(config_path="config", config_name="config")
def main_func(cfg: DictConfig):
    log.debug("Debug level message")
    log.info("Info level message")
    log.warning("Warning level message")
```

The log of `python main.py` in this case would be (in `main.log`)

```
[2021-03-13 17:36:06,493][__main__][INFO] - Info level message
[2021-03-13 17:36:06,493][__main__][WARNING] - Warning level message
```

If you want to include `DEBUG` also, then override `hydra.verbose=true` or `hydra.verbose=__main__` (i.e. `python main.py hydra.verbose=true`). The output in `main.log` in this case would be:

```
[2021-03-13 17:36:38,425][__main__][DEBUG] - Debug level message
[2021-03-13 17:36:38,425][__main__][INFO] - Info level message
[2021-03-13 17:36:38,425][__main__][WARNING] - Warning level message
```